In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("../bpepi/bpepi"))))
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("../bpepi"))))

import bpepi
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch


In [2]:
import networkx as nx
import numpy as np

In [3]:
import sys
sys.path.insert(0,'../Analysis')
sys.path.insert(0,'../bpepi')

In [4]:
from gen import simulate_one_detSIR
from bpepi.Modules import fg_torch as fg

In [5]:
torch.set_num_threads(2)

## Random Regular Graph

In [6]:
N=20000
d=3
lam=0.8
delta=0.01
rho=0.01
G=nx.random_regular_graph(d,N)
for (u,v) in G.edges():
    G.edges[u,v]['lambda'] = lam
if (N<500) : nx.draw(G)

In [7]:
snn = np.array(simulate_one_detSIR(G, s_type = "delta", S = delta, mask = ["SI"], T_max=100))

In [8]:
T = len(snn) - 1

In [9]:
from gen import generate_contacts
contacts = generate_contacts(G,T,lam)

#### Observations

In [10]:
from gen import generate_sensors_obs
num_obs = int(rho*N)
obs = generate_sensors_obs(snn, o_type="n_obs", M=num_obs)

In [11]:
tol=1e-4
damp=0.

In [12]:
def print_iter(err, it):
    print(f"\r err:{err:.2}, it:{it}", end="")

In [13]:
import torch
contacts_t = torch.tensor(contacts)

In [32]:
from bpepi.Modules import st_torch as st_p
from bpepi.Modules import st as st
import importlib
importlib.reload(st_p)
importlib.reload(st)

from bpepi.Modules import fg_torch as fg_p
from bpepi.Modules import fg as fg
import importlib
importlib.reload(fg)
importlib.reload(fg_p)

dtype=torch.float32
fg_p_ = fg_p.FactorGraph(N,T,contacts,obs,delta=delta, dtype=dtype)
fg_ = fg.FactorGraph(N,T,contacts,obs,delta=delta)

In [24]:
%timeit fg_.iterate(damp=damp)

178 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit fg_p_.iterate(damp=damp)

79 ms ± 376 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
fg_p_ = fg_p.FactorGraph(N,T,contacts,obs,delta=delta, dtype=dtype)
fg_ = fg.FactorGraph(N,T,contacts,obs,delta=delta)

In [33]:
fg_.update(tol=tol, print_iter=print_iter, damp=damp, maxit=1000)

 err:9.5e-05, it:47

(47, 9.506076407178599e-05)

In [34]:
fg_p_.update(tol=tol, print_iter=print_iter, damp=damp, maxit=1000)

 err:9.3e-05, it:47

(47, tensor(9.3073e-05))

In [35]:
(np.abs(fg_p_.messages.values.numpy() - fg_.messages.values)).mean()

2.1923592325971343e-08

In [36]:
marg = fg_.marginals()
marg_p = fg_p_.marginals().numpy()

In [37]:
(np.abs(marg - marg_p)).mean()

2.334497065292663e-07